In [1]:
import os
import eikon as ek
import numpy as np
import pickle
import pandas as pd
import cufflinks as cf
import configparser as cp

In [2]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

['eikon.cfg']

In [3]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## Get a List of CIKs

### From Compustat
#### For Both Bankrupt and Healthy Companies

In [4]:
# set the path to csv files
# path = 'D:\\studyproject\\bankruptcy\\data\\Compustat\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Compustat/' # for mac

# read the csv files
bankrupt_compustat = pd.read_csv(path + 'list_bankrupt.csv', dtype=object, index_col=0)
healthy = pd.read_csv(path + 'list_healthy.csv', dtype=object, index_col=0)

bankrupt_compustat.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132


In [5]:
healthy.head()

,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK
0,001004,AAR Corp,.,NaN,AIR,000361105,0000001750
1,001013,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478
2,001021,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126
3,001034,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469
4,001045,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201


In [6]:
bankrupt_compustat_CIK = bankrupt_compustat.CIK.dropna().to_list()
healthy_CIK = healthy.CIK.dropna().to_list()

print(len(bankrupt_compustat_CIK), 'CIKs available for Compustat bankrupt companies, and')
print(len(healthy_CIK), 'CIKs available for healthy companies.')


100 CIKs available for Compustat bankrupt companies, and
14761 CIKs available for healthy companies.


### From Edgar
#### For Bankrupt Companies

In [7]:
# set the path to csv file
# path = 'D:\\studyproject\\bankruptcy\\data\\Edgar\\' # for win
path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Edgar/' # for mac

# read the csv files
bankrupt_edgar = pd.read_csv(path + 'edgar_bankrupt_companies.csv', dtype=object)

bankrupt_edgar.head()

,CIK,Company name,Bankruptcy chapter,Form type,Date,URL
0,0000002034,ACETO CORP,11,8-K,09/20/2019,https://www.sec.gov/Archives/edgar/data/2034/0...
1,0000002178,"ADAMS RESOURCES & ENERGY, INC.",11,8-K,04/24/2017,https://www.sec.gov/Archives/edgar/data/2178/0...
2,0000004515,AMERICAN AIRLINES INC,11,8-K,11/29/2011,https://www.sec.gov/Archives/edgar/data/4515/0...
3,0000004707,AMERICA CAPITAL CORP,11,8-K,02/09/2009,https://www.sec.gov/Archives/edgar/data/4707/0...
4,0000006201,AMR CORP,11,8-K,10/23/2013,https://www.sec.gov/Archives/edgar/data/6201/0...


In [8]:
bankrupt_edgar_CIK = bankrupt_edgar.CIK.dropna().to_list()

print(len(bankrupt_edgar_CIK), 'CIKs available for Edgar bankrupt companies.')

1075 CIKs available for Edgar bankrupt companies.


## Merging Bankrupt Lists

In [9]:
# add two lists and remove the duplicates
bankrupt_CIK = list(set(bankrupt_compustat_CIK + bankrupt_edgar_CIK))

len(bankrupt_CIK)

1103

****

## Download the Data

In [10]:
# create folders for saving data
if not os.path.exists('data_bankrupt_CIK'):
    os.makedirs('data_bankrupt_CIK')
    
if not os.path.exists('data_healthy_CIK'):
    os.makedirs('data_healthy_CIK')

In [11]:
# Eikon does not work with large lists, so we
# need to break the healthy_CIK list
# and also bankrupt_CIK

bankrupt_CIK_list = [bankrupt_CIK[i:i + 500] for i in range(0, len(bankrupt_CIK), 500)]
healthy_CIK_list = [healthy_CIK[i:i + 500] for i in range(0, len(healthy_CIK), 500)]

print(len(bankrupt_CIK_list))
len(healthy_CIK_list)


3


30

### Financial Factors


In [12]:
# add all financial factors we need to replicate *the paper*

fields = ['TR.CommonName',
          'TR.CIKNUMBER',
          'TR.TotalCurrentAssets', # Current Assets
          'TR.TotalCurrLiabilities', # Current Liabilities
          'TR.AccountsPayableCFStmt', # Accounts Payable
          'TR.CashAndSTInvestments',
          'TR.TotalAssetsReported',
          'TR.CashAndSTInvestments',
          'TR.MarketCapDS',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.DepreciationAmort',
          'TR.TotalAssetsReported',
          'TR.TotalDebtOutstanding',
          'TR.NetDebt',
          'TR.TotalAssetsReported',
          'TR.Inventories',
          'TR.NetSales',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.TotalLiabilities',
          'TR.TotalCurrLiabilities',
          'TR.NetSales',
          'TR.TotalLiabilities',
          'TR.TotalAssetsReported',
          'TR.TotalLiabilities',
          'TR.TotalLiabilities',
          'TR.TangibleBookValueRptd',
          'TR.TangibleBookValueBnk',
          'TR.TangibleBookValueIns',
          'TR.TangibleBookValueUtil',
          'TR.TangibleBVPS',
          'TR.NetIncomeStartingLine',
          'TR.TotalAssetsReported',
          'TR.NetIncomeStartingLine',
          'TR.NetSales',
          'TR.OperatingIncome',
          'TR.TotalAssetsReported',
          'TR.TotalCurrLiabilities',
          'TR.RetainedEarnings',
          'TR.TotalAssetsReported',
          'TR.TotalEquity',
          'TR.TotalAssetsReported',
          'TR.InstrumentIsActive', # bankruptcy check (not reliable)
          'TR.IsDelistedQuote',
          'TR.NAICSIndustryGroup']  # bankruptcy check (not reliable)


print(len(fields))

# remove duplicates
fields = list(set(fields))

print(len(fields))

49
26


# Download Data (For Real This Time)

In [13]:
def get_data(CIK_list, status, year, month, day, chunk):
    df, err = ek.get_data(CIK_list[chunk], fields=fields,
                          parameters={'Frq':'Y',
                                      'SDate':'{}-01-01'.format(year),
                                      'EDate':'{}-{}-{}'.format(year, month, day)})
    df.to_csv('data_{}_CIK/{}_data{}_{}.csv'.format(status, status, chunk+1, year), index=False)

In [14]:
get_data(bankrupt_CIK_list, 'bankrupt', '2000', '12', '30', 0)

In [15]:
get_data(bankrupt_CIK_list, 'bankrupt', '2000', '12', '30', 1)

In [16]:
get_data(bankrupt_CIK_list, 'bankrupt', '2000', '12', '30', 2)

In [17]:
get_data(bankrupt_CIK_list, 'bankrupt', '2001', '12', '30', 0)

In [19]:
get_data(bankrupt_CIK_list, 'bankrupt', '2001', '12', '30', 1)

In [20]:
get_data(bankrupt_CIK_list, 'bankrupt', '2001', '12', '30', 2)

In [21]:
get_data(bankrupt_CIK_list, 'bankrupt', '2002', '12', '30', 0)

In [22]:
get_data(bankrupt_CIK_list, 'bankrupt', '2002', '12', '30', 1)

In [23]:
get_data(bankrupt_CIK_list, 'bankrupt', '2002', '12', '30', 2)

In [24]:
get_data(bankrupt_CIK_list, 'bankrupt', '2003', '12', '30', 0)

In [25]:
get_data(bankrupt_CIK_list, 'bankrupt', '2003', '12', '30', 1)

In [26]:
get_data(bankrupt_CIK_list, 'bankrupt', '2003', '12', '30', 2)

In [27]:
get_data(bankrupt_CIK_list, 'bankrupt', '2004', '12', '30', 0)

In [28]:
get_data(bankrupt_CIK_list, 'bankrupt', '2004', '12', '30', 1)

In [29]:
get_data(bankrupt_CIK_list, 'bankrupt', '2004', '12', '30', 2)

In [30]:
get_data(bankrupt_CIK_list, 'bankrupt', '2005', '12', '30', 0)

In [31]:
get_data(bankrupt_CIK_list, 'bankrupt', '2005', '12', '30', 1)

In [32]:
get_data(bankrupt_CIK_list, 'bankrupt', '2005', '12', '30', 2)

In [33]:
get_data(bankrupt_CIK_list, 'bankrupt', '2006', '12', '30', 0)

In [34]:
get_data(bankrupt_CIK_list, 'bankrupt', '2006', '12', '30', 1)

In [35]:
get_data(bankrupt_CIK_list, 'bankrupt', '2006', '12', '30', 2)

In [36]:
get_data(bankrupt_CIK_list, 'bankrupt', '2007', '12', '30', 0)

In [37]:
get_data(bankrupt_CIK_list, 'bankrupt', '2007', '12', '30', 1)

In [38]:
get_data(bankrupt_CIK_list, 'bankrupt', '2007', '12', '30', 2)

In [39]:
get_data(bankrupt_CIK_list, 'bankrupt', '2008', '12', '30', 0)

In [40]:
get_data(bankrupt_CIK_list, 'bankrupt', '2008', '12', '30', 1)

In [41]:
get_data(bankrupt_CIK_list, 'bankrupt', '2008', '12', '30', 2)

In [42]:
get_data(bankrupt_CIK_list, 'bankrupt', '2009', '12', '30', 0)

In [43]:
get_data(bankrupt_CIK_list, 'bankrupt', '2009', '12', '30', 1)

In [44]:
get_data(bankrupt_CIK_list, 'bankrupt', '2009', '12', '30', 2)

In [45]:
get_data(bankrupt_CIK_list, 'bankrupt', '2010', '12', '30', 0)

In [46]:
get_data(bankrupt_CIK_list, 'bankrupt', '2010', '12', '30', 1)

In [47]:
get_data(bankrupt_CIK_list, 'bankrupt', '2010', '12', '30', 2)

In [48]:
get_data(bankrupt_CIK_list, 'bankrupt', '2011', '12', '30', 0)

In [51]:
get_data(bankrupt_CIK_list, 'bankrupt', '2011', '12', '30', 1)

In [52]:
get_data(bankrupt_CIK_list, 'bankrupt', '2011', '12', '30', 2)

In [53]:
get_data(bankrupt_CIK_list, 'bankrupt', '2012', '12', '30', 0)

In [54]:
get_data(bankrupt_CIK_list, 'bankrupt', '2012', '12', '30', 1)

In [55]:
get_data(bankrupt_CIK_list, 'bankrupt', '2012', '12', '30', 2)

In [56]:
get_data(bankrupt_CIK_list, 'bankrupt', '2013', '12', '30', 0)

In [57]:
get_data(bankrupt_CIK_list, 'bankrupt', '2013', '12', '30', 1)

In [58]:
get_data(bankrupt_CIK_list, 'bankrupt', '2013', '12', '30', 2)

In [59]:
get_data(bankrupt_CIK_list, 'bankrupt', '2014', '12', '30', 0)

In [60]:
get_data(bankrupt_CIK_list, 'bankrupt', '2014', '12', '30', 1)

In [61]:
get_data(bankrupt_CIK_list, 'bankrupt', '2014', '12', '30', 2)

In [62]:
get_data(bankrupt_CIK_list, 'bankrupt', '2015', '12', '30', 0)

In [63]:
get_data(bankrupt_CIK_list, 'bankrupt', '2015', '12', '30', 1)

In [64]:
get_data(bankrupt_CIK_list, 'bankrupt', '2015', '12', '30', 2)

In [65]:
get_data(bankrupt_CIK_list, 'bankrupt', '2016', '12', '30', 0)

In [66]:
get_data(bankrupt_CIK_list, 'bankrupt', '2016', '12', '30', 1)

In [67]:
get_data(bankrupt_CIK_list, 'bankrupt', '2016', '12', '30', 2)

In [68]:
get_data(bankrupt_CIK_list, 'bankrupt', '2017', '12', '30', 0)

In [69]:
get_data(bankrupt_CIK_list, 'bankrupt', '2017', '12', '30', 1)

In [70]:
get_data(bankrupt_CIK_list, 'bankrupt', '2017', '12', '30', 2)

In [71]:
get_data(bankrupt_CIK_list, 'bankrupt', '2018', '12', '30', 0)

In [72]:
get_data(bankrupt_CIK_list, 'bankrupt', '2018', '12', '30', 1)

In [73]:
get_data(bankrupt_CIK_list, 'bankrupt', '2018', '12', '30', 2)

In [74]:
get_data(bankrupt_CIK_list, 'bankrupt', '2019', '12', '30', 0)

In [75]:
get_data(bankrupt_CIK_list, 'bankrupt', '2019', '12', '30', 1)

In [76]:
get_data(bankrupt_CIK_list, 'bankrupt', '2019', '12', '30', 2)

****

In [77]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 0)

In [78]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 1)

In [79]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 2)

In [80]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 3)

In [81]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 4)

In [82]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 5)

In [83]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 6)

In [84]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 7)

In [85]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 8)

In [86]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 9)

In [87]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 10)

In [88]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 11)

In [89]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 12)

In [90]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 13)

In [91]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 14)

In [92]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 15)

In [93]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 16)

In [94]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 17)

In [95]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 18)

In [96]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 19)

In [97]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 20)

In [98]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 21)

In [99]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 22)

In [100]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 23)

In [101]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 24)

In [102]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 25)

In [103]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 26)

In [104]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 27)

In [105]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 28)

In [106]:
get_data(healthy_CIK_list, 'healthy', '2000', '12', '30', 29)

In [107]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 0)

In [108]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 1)

In [109]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 2)

In [110]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 3)

In [111]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 4)

In [112]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 5)

In [113]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 6)

In [114]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 7)

In [115]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 8)

In [116]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 9)

In [117]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 10)

In [118]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 11)

In [119]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 12)

In [120]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 13)

In [121]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 14)

In [122]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 15)

In [123]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 16)

In [124]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 17)

In [125]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 18)

In [126]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 19)

In [127]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 20)

In [128]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 21)

In [129]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 22)

In [130]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 23)

In [131]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 24)

In [132]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 25)

In [133]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 26)

In [134]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 27)

In [135]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 28)

In [136]:
get_data(healthy_CIK_list, 'healthy', '2001', '12', '30', 29)

In [137]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 0)

In [138]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 1)

In [139]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 2)

In [140]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 3)

In [141]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 4)

In [142]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 5)

In [143]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 6)

In [144]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 7)

In [145]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 8)

In [146]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 9)

In [147]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 10)

In [148]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 11)

In [149]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 12)

In [150]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 13)

In [151]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 14)

In [152]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 15)

In [153]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 16)

In [154]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 17)

In [155]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 18)

In [156]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 19)

In [157]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 20)

In [158]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 21)

In [159]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 22)

In [160]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 23)

In [161]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 24)

In [162]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 25)

In [163]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 26)

In [164]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 27)

In [165]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 28)

In [166]:
get_data(healthy_CIK_list, 'healthy', '2002', '12', '30', 29)

In [167]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 0)

In [168]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 1)

In [169]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 2)

In [170]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 3)

In [171]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 4)

In [172]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 5)

In [173]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 6)

In [174]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 7)

In [175]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 8)

In [176]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 9)

In [177]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 10)

In [178]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 11)

In [179]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 12)

In [180]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 13)

In [181]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 14)

In [182]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 15)

In [183]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 16)

In [184]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 17)

In [185]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 18)

In [186]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 19)

In [187]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 20)

In [188]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 21)

In [189]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 22)

In [190]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 23)

In [191]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 24)

In [192]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 25)

In [193]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 26)

In [194]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 27)

In [195]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 28)

In [196]:
get_data(healthy_CIK_list, 'healthy', '2003', '12', '30', 29)

In [197]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 0)

In [198]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 1)

In [199]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 2)

In [200]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 3)

In [201]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 4)

In [202]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 5)

In [203]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 6)

In [204]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 7)

In [205]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 8)

In [206]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 9)

In [207]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 10)

In [208]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 11)

In [209]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 12)

In [210]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 13)

In [211]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 14)

In [212]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 15)

In [213]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 16)

In [214]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 17)

In [215]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 18)

In [216]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 19)

In [217]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 20)

In [218]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 21)

In [219]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 22)

In [220]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 23)

In [221]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 24)

In [222]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 25)

In [223]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 26)

In [224]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 27)

In [225]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 28)

In [226]:
get_data(healthy_CIK_list, 'healthy', '2004', '12', '30', 29)

In [227]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 0)

In [228]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 1)

In [229]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 2)

In [230]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 3)

In [231]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 4)

In [232]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 5)

In [233]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 6)

In [234]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 7)

In [235]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 8)

In [236]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 9)

In [237]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 10)

In [238]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 11)

In [239]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 12)

In [240]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 13)

In [241]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 14)

In [242]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 15)

In [243]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 16)

In [244]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 17)

In [245]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 18)

In [246]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 19)

In [247]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 20)

In [248]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 21)

In [249]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 22)

In [250]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 23)

In [251]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 24)

In [252]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 25)

In [253]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 26)

In [254]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 27)

In [255]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 28)

In [256]:
get_data(healthy_CIK_list, 'healthy', '2005', '12', '30', 29)

In [257]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 0)

In [258]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 1)

In [259]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 2)

In [260]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 3)

In [261]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 4)

In [262]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 5)

In [263]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 6)

In [264]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 7)

In [265]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 8)

In [266]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 9)

In [267]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 10)

In [268]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 11)

In [269]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 12)

In [270]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 13)

In [271]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 14)

In [272]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 15)

In [273]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 16)

In [274]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 17)

In [275]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 18)

In [276]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 19)

In [277]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 20)

In [278]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 21)

In [279]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 22)

In [280]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 23)

In [281]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 24)

In [282]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 25)

In [283]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 26)

In [284]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 27)

In [285]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 28)

In [286]:
get_data(healthy_CIK_list, 'healthy', '2006', '12', '30', 29)

In [287]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 0)

In [288]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 1)

In [289]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 2)

In [290]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 3)

In [291]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 4)

In [292]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 5)

In [293]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 6)

In [294]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 7)

In [295]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 8)

In [296]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 9)

In [297]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 10)

In [298]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 11)

In [299]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 12)

In [300]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 13)

In [301]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 14)

In [302]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 15)

In [303]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 16)

In [304]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 17)

In [305]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 18)

In [306]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 19)

In [307]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 20)

In [308]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 21)

In [309]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 22)

In [310]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 23)

In [311]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 24)

In [312]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 25)

In [313]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 26)

In [314]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 27)

In [315]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 28)

In [316]:
get_data(healthy_CIK_list, 'healthy', '2007', '12', '30', 29)

In [317]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 0)

In [318]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 1)

In [319]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 2)

In [320]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 3)

In [321]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 4)

In [322]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 5)

In [323]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 6)

In [324]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 7)

In [325]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 8)

In [326]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 9)

In [327]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 10)

In [328]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 11)

In [329]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 12)

In [330]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 13)

In [331]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 14)

In [332]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 15)

In [333]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 16)

In [334]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 17)

In [335]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 18)

In [336]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 19)

In [337]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 20)

In [338]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 21)

In [339]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 22)

In [340]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 23)

In [341]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 24)

In [342]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 25)

In [343]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 26)

In [344]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 27)

In [345]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 28)

In [346]:
get_data(healthy_CIK_list, 'healthy', '2008', '12', '30', 29)

In [347]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 0)

In [348]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 1)

In [349]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 2)

In [350]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 3)

In [351]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 4)

In [352]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 5)

In [353]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 6)

In [354]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 7)

In [355]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 8)

In [356]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 9)

In [357]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 10)

In [358]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 11)

In [359]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 12)

In [360]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 13)

In [361]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 14)

In [362]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 15)

In [363]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 16)

In [364]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 17)

In [365]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 18)

In [366]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 19)

In [367]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 20)

In [368]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 21)

In [369]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 22)

In [370]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 23)

In [371]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 24)

In [372]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 25)

In [373]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 26)

In [374]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 27)

In [375]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 28)

In [376]:
get_data(healthy_CIK_list, 'healthy', '2009', '12', '30', 29)

In [377]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 0)

In [378]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 1)

In [379]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 2)

In [380]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 3)

In [381]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 4)

In [382]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 5)

In [383]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 6)

In [384]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 7)

In [385]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 8)

In [386]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 9)

In [387]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 10)

In [388]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 11)

In [389]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 12)

In [390]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 13)

In [391]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 14)

In [392]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 15)

In [393]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 16)

In [394]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 17)

In [395]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 18)

In [396]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 19)

In [397]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 20)

In [398]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 21)

In [399]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 22)

In [400]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 23)

In [401]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 24)

In [402]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 25)

In [403]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 26)

In [404]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 27)

In [405]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 28)

In [406]:
get_data(healthy_CIK_list, 'healthy', '2010', '12', '30', 29)

In [407]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 0)

In [408]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 1)

In [409]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 2)

In [410]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 3)

In [411]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 4)

In [412]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 5)

In [413]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 6)

In [414]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 7)

In [415]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 8)

In [416]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 9)

In [417]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 10)

In [418]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 11)

In [419]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 12)

In [420]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 13)

In [421]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 14)

In [422]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 15)

In [423]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 16)

In [424]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 17)

In [425]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 18)

In [426]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 19)

In [427]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 20)

In [428]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 21)

In [429]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 22)

In [430]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 23)

In [431]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 24)

In [432]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 25)

In [433]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 26)

In [434]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 27)

In [435]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 28)

In [436]:
get_data(healthy_CIK_list, 'healthy', '2011', '12', '30', 29)

In [437]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 0)

In [438]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 1)

In [439]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 2)

In [440]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 3)

In [441]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 4)

In [442]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 5)

In [443]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 6)

In [444]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 7)

In [445]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 8)

In [446]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 9)

In [447]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 10)

In [448]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 11)

In [449]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 12)

In [450]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 13)

In [451]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 14)

In [452]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 15)

In [453]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 16)

In [454]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 17)

In [455]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 18)

In [456]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 19)

In [457]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 20)

In [458]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 21)

In [459]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 22)

In [460]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 23)

In [461]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 24)

In [462]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 25)

In [463]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 26)

In [464]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 27)

In [465]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 28)

In [466]:
get_data(healthy_CIK_list, 'healthy', '2012', '12', '30', 29)

In [467]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 0)

In [468]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 1)

In [469]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 2)

In [470]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 3)

In [471]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 4)

In [472]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 5)

In [473]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 6)

In [474]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 7)

In [475]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 8)

In [476]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 9)

In [477]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 10)

In [478]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 11)

In [479]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 12)

In [480]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 13)

In [481]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 14)

In [482]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 15)

In [483]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 16)

In [484]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 17)

In [485]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 18)

In [486]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 19)

In [487]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 20)

In [488]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 21)

In [489]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 22)

In [490]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 23)

In [491]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 24)

In [492]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 25)

In [493]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 26)

In [494]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 27)

In [495]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 28)

In [496]:
get_data(healthy_CIK_list, 'healthy', '2013', '12', '30', 29)

In [497]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 0)

In [498]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 1)

In [499]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 2)

In [500]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 3)

In [501]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 4)

In [502]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 5)

In [503]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 6)

In [504]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 7)

In [505]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 8)

In [506]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 9)

In [507]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 10)

In [508]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 11)

In [509]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 12)

In [510]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 13)

In [511]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 14)

In [512]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 15)

In [513]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 16)

In [514]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 17)

In [515]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 18)

In [516]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 19)

In [517]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 20)

In [518]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 21)

In [519]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 22)

In [520]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 23)

In [521]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 24)

In [522]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 25)

In [523]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 26)

In [524]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 27)

In [525]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 28)

In [526]:
get_data(healthy_CIK_list, 'healthy', '2014', '12', '30', 29)

In [527]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 0)

In [528]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 1)

In [529]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 2)

In [530]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 3)

In [531]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 4)

In [532]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 5)

In [533]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 6)

In [534]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 7)

In [535]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 8)

In [536]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 9)

In [537]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 10)

In [538]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 11)

In [539]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 12)

In [540]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 13)

In [541]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 14)

In [542]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 15)

In [543]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 16)

In [544]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 17)

In [545]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 18)

In [546]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 19)

In [547]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 20)

In [548]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 21)

In [549]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 22)

In [550]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 23)

In [551]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 24)

In [552]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 25)

In [553]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 26)

In [554]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 27)

In [555]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 28)

In [556]:
get_data(healthy_CIK_list, 'healthy', '2015', '12', '30', 29)

In [557]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 0)

In [558]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 1)

In [559]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 2)

In [560]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 3)

In [561]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 4)

In [562]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 5)

In [563]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 6)

In [564]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 7)

In [565]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 8)

In [566]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 9)

In [567]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 10)

In [568]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 11)

In [569]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 12)

In [570]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 13)

In [571]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 14)

In [572]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 15)

In [573]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 16)

In [574]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 17)

In [575]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 18)

In [576]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 19)

In [577]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 20)

In [578]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 21)

In [579]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 22)

In [580]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 23)

In [581]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 24)

In [582]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 25)

In [583]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 26)

In [584]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 27)

In [585]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 28)

In [586]:
get_data(healthy_CIK_list, 'healthy', '2016', '12', '30', 29)

In [587]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 0)

In [588]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 1)

In [589]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 2)

In [590]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 3)

In [591]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 4)

In [592]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 5)

In [593]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 6)

In [594]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 7)

In [595]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 8)

In [596]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 9)

In [597]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 10)

In [598]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 11)

In [599]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 12)

In [600]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 13)

In [601]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 14)

In [602]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 15)

In [603]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 16)

In [604]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 17)

In [605]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 18)

In [606]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 19)

In [607]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 20)

In [608]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 21)

In [609]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 22)

In [610]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 23)

In [611]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 24)

In [612]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 25)

In [613]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 26)

In [614]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 27)

In [615]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 28)

In [616]:
get_data(healthy_CIK_list, 'healthy', '2017', '12', '30', 29)

In [617]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 0)

In [618]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 1)

In [619]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 2)

In [620]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 3)

In [621]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 4)

In [622]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 5)

In [623]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 6)

In [624]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 7)

In [625]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 8)

In [626]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 9)

In [627]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 10)

In [628]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 11)

In [629]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 12)

In [630]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 13)

In [631]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 14)

In [632]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 15)

In [633]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 16)

In [634]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 17)

In [635]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 18)

In [636]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 19)

In [637]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 20)

In [638]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 21)

In [639]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 22)

In [640]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 23)

In [641]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 24)

In [642]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 25)

In [643]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 26)

In [644]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 27)

In [645]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 28)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2018', '12', '30', 29)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 0)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 1)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 2)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 3)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 4)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 5)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 6)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 7)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 8)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 9)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 10)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 11)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 12)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 13)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 14)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 15)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 16)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 17)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 18)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 19)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 20)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 21)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 22)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 23)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 24)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 25)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 26)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 27)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 28)

In [ ]:
get_data(healthy_CIK_list, 'healthy', '2019', '12', '30', 29)